Import Libraries


In [2]:
import numpy as np
from osgeo import gdal
from scipy import stats

<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

Load test data

In [ ]:
r = gdal.Open("data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif")
g = np.array(r.GetRasterBand(1).ReadAsArray())

In [24]:
##stats.describe(g) 
out = np.zeros(g.shape,dtype='uint64')

95


Function to create data structure function takes in a grid with mxn and returns a grid with mxnxc where c is the number of classes.  A vectorized version would be faster

In [46]:
def dtFZS(g,out):
    for xw in range(1,g.shape[0]):
        for yw in range(1,g.shape[1]):
            out[xw,yw] = out[xw,yw-1] + g[xw,yw]
    print("done")

In [47]:
##out = dtFZS(g)
out = np.zeros(g.shape,dtype='uint64')
dtFZS(g,out)


done


In [49]:
stats.describe(out)
##out.shape

DescribeResult(nobs=1734, minmax=(array([0, 0, 0, ..., 0, 0, 0], dtype=uint64), array([     0,     95,    190, ..., 158553, 158634, 158716], dtype=uint64)), mean=array([  0.00000000e+00,   5.19550173e+01,   1.04167820e+02, ...,
         1.38627170e+05,   1.38692304e+05,   1.38757807e+05]), variance=array([  0.00000000e+00,   6.99059718e+02,   2.48874562e+03, ...,
         9.65962039e+07,   9.66710595e+07,   9.67510685e+07]), skewness=array([ 0.        ,  0.06241906,  0.04223732, ..., -1.76924598,
       -1.76931426, -1.76938001]), kurtosis=array([ -3.        ,  -1.62990199,  -1.48996093, ...,  21.67019242,
        21.67734873,  21.68228947]))

Fast Summary Statistics Function
    

In [54]:
def FstZS(v,g):
    total = 0;
    ## If function moves left add above if right substract above
    for p in range(1,v.size):
        ## If moving left
        if v.x[p] - v.x[p-1] > 0:
            total += g[v.x[p],v.y[p]]
        ## If moving Right
        elif v.x[p] - v.x[p-1] < 0:
            total -= g[v.x[p],v.y[p]]
            

    
    
    
    
    
    
    
    

In [ ]:
v = 